# Challenge 1

I attempted to use aprils data too to try to predict junes, however it considerably lowered the quality of the model: 

Loss: 0.0002993

MAE: 0.005401

RMSE: 0.01730

R²: 0.6074

Due to this, I am no longer using the models for it. However, may is an uncertain choice for predicting july's data. 

I also ruled out using relu, as it did not yield the results of tanh. 

In [11]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder, StandardScaler

import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import Sequence 
from tensorflow.keras.optimizers import Adam    
from sklearn.model_selection import train_test_split, TimeSeriesSplit
# from tensorflow.keras.losses import Huber

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score



# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,  mean_squared_log_error


In [2]:
df_omie_labelled = pd.read_csv('data\\df_omie_labelled.csv')
unit_list = pd.read_csv('data\\unit_list.csv')
filtered_categories = pd.read_csv('data\\filtered_categories.csv')
df_omie_blind = pd.read_csv('data\\df_omie_blind.csv')

df_omie_labelled['fechaHora'] = pd.to_datetime(df_omie_labelled['fechaHora'])

df = pd.merge(df_omie_labelled, filtered_categories, on='Codigo', how='left')
df = pd.merge(df, unit_list, on='Codigo', how='left')

df.head()

,Codigo,Descripcion,fechaHora,PrecEuro,Energia,Categoria,Descripción,Agente,Porcentaje_Propiedad,Tipo_Unidad,Zona/Frontera,Tecnología
0,ADXRE01,PREAL1,2024-02-29,0.0,0.0,Solar,PREAL1,AUDAX RENOVABLES S.A.,100.0,GENERACION,ZONA ESPAÑOLA,RE Mercado Solar Fotovoltáica
1,ADXVD03,PV ALARCOS,2024-02-29,0.0,0.0,Solar,PV ALARCOS,AUDAX RENOVABLES S.A.,100.0,GENERACION,ZONA ESPAÑOLA,RE Mercado Solar Fotovoltáica
2,ADXVD04,EOLICA AUDAX,2024-02-29,-0.1,2.7,Eolica,EOLICA AUDAX,AUDAX RENOVABLES S.A.,100.0,GENERACION,ZONA ESPAÑOLA,RE Mercado Eólica
3,AIRVD01,AIRVD01,2024-02-29,0.0,0.0,Solar,AIRVD01,AIRE LIMPIO S.L.,100.0,GENERACION,ZONA ESPAÑOLA,RE Mercado Solar Fotovoltáica
4,AIRVD02,AIRVD02,2024-02-29,0.0,0.0,Solar,AIRVD02,AIRE LIMPIO S.L.,100.0,GENERACION,ZONA ESPAÑOLA,RE Mercado Solar Fotovoltáica


In [3]:
print(df['Energia'].describe())
print(df[df['Energia'] < 1])  

count    2.409526e+06
mean     1.386135e+01
std      7.212333e+01
min      0.000000e+00
25%      0.000000e+00
50%      8.000000e-01
75%      9.100000e+00
max      5.047000e+03
Name: Energia, dtype: float64
          Codigo                     Descripcion           fechaHora  \
0        ADXRE01                          PREAL1 2024-02-29 00:00:00   
1        ADXVD03                      PV ALARCOS 2024-02-29 00:00:00   
3        AIRVD01                         AIRVD01 2024-02-29 00:00:00   
4        AIRVD02                         AIRVD02 2024-02-29 00:00:00   
5        ALDRE01                ALDRO GENERACION 2024-02-29 00:00:00   
...          ...                             ...                 ...   
2409516  WMVD155  VENTA EXCED AUTOPROD WMARK 155 2024-06-01 22:00:00   
2409517  WMVD166  VENTA EXCED AUTOPROD WMARK 166 2024-06-01 22:00:00   
2409522  WMVD206  VENTA EXCED AUTOPROD WMARK 206 2024-06-01 22:00:00   
2409524  ZRMRE01                  UOF CERVANTINA 2024-06-01 22:00:00   
24

# LSTM

## Prepare Data

In [87]:
def prepare_data(df, sequence_length=24, test_size=1, exogenous_features=['PrecEuro']):
    df_copy = df.copy()

    df_copy['log_Energia'] = np.log1p(df_copy['Energia'])
    
    df_copy['fechaHora'] = pd.to_datetime(df_copy['fechaHora'])
    df_copy['Hora'] = df_copy['fechaHora'].dt.hour
    df_copy['DiaSemana'] = df_copy['fechaHora'].dt.weekday
    df_copy['Mes'] = df_copy['fechaHora'].dt.month

    categorical_columns = ['Categoria', 'Agente', 'Tipo_Unidad', 'Zona/Frontera', 'Tecnología']
    df_copy = pd.get_dummies(df_copy, columns=categorical_columns, drop_first=True)
    
    features = ['Categoria', 'Tecnología', 'Hora', 'DiaSemana', 'Mes'] + exogenous_features

    # numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

    #fit scaler only on training data    
    scaler = StandardScaler()
    feature_columns = [col for col in df_copy.columns if col not in ['Fecha', 'Energia', 'log_Energia', 'Codigo', 'fechaHora']]
    df_copy[feature_columns] = scaler.fit_transform(df_copy[feature_columns])
    # df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

    return df, scaler

In [64]:
class EnergySequence(Sequence): 
    def __init__(self, df, target_col, sequence_length=24, batch_size=32, n_plants=300,  is_test=False):
        self.df = df
        self.target_col = target_col
        self.sequence_length = sequence_length
        self.batch_size = batch_size
        self.n_plants = n_plants
        self.is_test = is_test
        self.plants = df['Codigo'].unique()
         

    # def __len__(self):
    #     return (len(self.data) - self.sequence_length) // self.batch_size 
    
    def __len__(self):
        return int(np.floor(len(self.df)/ self.batch_size))

    


    def __getitem__(self, idx):
        plant_code = self.plants[idx % len(self.plants)]
        plant_data = self.df[self.df['Codigo'] == plant_code]
        batch_features = []
        batch_target = []
   

        for i in range(self.sequence_length, len(plant_data)):
            if i + self.sequence_length < len(self.df):
                sequence = self.df.iloc[i : i + self.sequence_length]
                features = sequence.drop(columns=['Energia', 'log_Energia', 'fechaHora'])
                target = sequence['log_Energia'].iloc[-1]  
                
                batch_features.append(features.values)
                batch_target.append(target)
                    
        return np.array(batch_features), np.array(batch_target)


##  Build Models 
(relu, tanh, combination)

In [48]:
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, activation='tanh',return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32, activation='tanh',),
        Dropout(0.2),
        # Dense(units[2], activation=activations[2]),
        Dense(1)
    ])

    optimizer = Adam(learning_rate=0.0005)  
    model.compile(optimizer=optimizer, loss='mse')

    return model

## Train & Evaluate Models

trains on april-may, may data to predict junes data. From here I will determine which model (am/m using relu/tanh/combination) is best to predict july

In [49]:
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(actual - predicted) / (np.abs(actual) + np.abs(predicted) + 1e-8))  


def evaluate_model(model, test_generator):
    # train_generator = EnergySequence(train_data[['Energia', 'PrecEuro']], target_col=0, sequence_length=sequence_length, batch_size=batch_size)
    # test_generator = EnergySequence(test_data[['Energia', 'PrecEuro']], target_col=0, sequence_length=sequence_length, batch_size=batch_size)

    # history = model.fit(train_generator, validation_data=test_generator, epochs=epochs, verbose=1)
    # loss = model.evaluate(test_generator)
    
    predictions = model.predict(test_generator, verbose=0)
    # true_values = test_generator.df['log_Energia'].iloc[test_generator.indexes[-len(predictions):]].values
    true_values = test_generator.data['log_Energia'].iloc[-len(predictions):].values

    predictions = np.expm1(predictions)
    true_values = np.expm1(true_values)

    # test_targets = test_data['Energia'].iloc[sequence_length:sequence_length+len(predictions)].values
    # test_dates = test_data['fechaHora'].iloc[sequence_length:sequence_length+len(predictions)].values
    
    # error metrics
    mae = mean_absolute_error(true_values, predictions)
    rmse = np.sqrt(mean_squared_error(true_values, predictions))
    r2 = r2_score(true_values, predictions)
    mape = smape(true_values, predictions.flatten())

    return {
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "MAPE": mape,
        "Predictions": predictions.flatten(),  
    }


In [88]:
# def cross_validate_model(df, n_splits=3, sequence_length=24):
#     tscv = TimeSeriesSplit(n_splits=n_splits)
#     for train_idx, test_idx in tscv.split(df):
#         train_data, test_data = df.iloc[train_idx], df.iloc[test_idx]
        
#         # train_generator = EnergySequence(train_data, sequence_length=sequence_length)
#         train_generator = EnergySequence(train_data, target_col='log_Energia', sequence_length=sequence_length, batch_size=32, n_plants=300)
#         test_generator = EnergySequence(test_data, target_col='log_Energia', sequence_length=sequence_length, is_test=True)
        
#         model = build_lstm_model((train_generator.sequence_length, train_data.shape[1] - 3))
        
#         model.fit(train_generator, epochs=5, batch_size=32, verbose=1)
        
#         mae, rmse, r2, smape = evaluate_model(model, test_generator)
#         print(f"MAE: {mae}, RMSE: {rmse}, R2: {r2}, SMAPE: {smape}")

def cross_validate_model(df, n_splits=3, sequence_length=24):
    df_copy = df.copy()
    df_copy = df_copy.drop(columns=['Descripcion', 'fechaHora'])
    
    # One-hot encode categorical columns
    categorical_columns = ['Categoria', 'Agente', 'Tipo_Unidad', 'Zona/Frontera', 'Tecnología']
    df_copy = pd.get_dummies(df_copy, columns=categorical_columns, drop_first=True)
    
    # Keep 'Codigo' for prediction, but exclude it from the feature set
    feature_columns = [col for col in df_copy.columns if col not in ['Energia', 'log_Energia', 'Codigo']]  # Exclude target and 'Codigo'
    
    # Handle time-series split and training
    tscv = TimeSeriesSplit(n_splits=n_splits)
    for train_idx, test_idx in tscv.split(df_copy):
        train_data, test_data = df_copy.iloc[train_idx], df_copy.iloc[test_idx]
        
        # Make sure only numeric columns are used for features
        train_data = train_data.select_dtypes(include=[np.number])
        test_data = test_data.select_dtypes(include=[np.number])
        
        # Now, create the generators
        train_generator = EnergySequence(train_data, target_col='log_Energia', sequence_length=sequence_length, batch_size=32, n_plants=300)
        test_generator = EnergySequence(test_data, target_col='log_Energia', sequence_length=sequence_length, is_test=True)
        
        # Build and train the model
        model = build_lstm_model((train_generator.sequence_length, train_data.shape[1] - 3))
        model.fit(train_generator, epochs=5, batch_size=32, verbose=1)
        
        # Evaluate the model
        mae, rmse, r2, smape = evaluate_model(model, test_generator)
        print(f"MAE: {mae}, RMSE: {rmse}, R2: {r2}, SMAPE: {smape}")
        
        # After prediction, reattach 'Codigo' to the results for prediction purposes
        predicted_values = model.predict(test_generator)
        test_data['Predicted_Energia'] = predicted_values
        
        # Keep 'Codigo' in the final prediction output
        predictions = test_data[['Codigo', 'Predicted_Energia']]
        print(predictions)



print(df.columns)
prepared_df, scaler = prepare_data(df, sequence_length=24, test_size=1, exogenous_features=['PrecEuro'])

sequence_length = 24
X = df.drop(columns=['Energia', 'Codigo', 'fechaHora'])  
y = df['Energia'] 
print("Shape of X before reshaping:", X.shape)
print("Shape of y before slicing:", y.shape)
X = X.select_dtypes(include=[np.number])


# numeric_columns = X.select_dtypes(include=[np.number]).columns

# scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


n_samples = X_scaled.shape[0] // sequence_length
X= X_scaled[:n_samples * sequence_length]
X= X.reshape((n_samples, sequence_length, X.shape[1]))
y= y.iloc[sequence_length-1:n_samples * sequence_length:sequence_length].values

print("Shape of X:", X.shape)  # Expected: (100396, 24, 9)    
print("Shape of y:", y.shape)  # Expected: (100396,) 
   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)  
           
print("X_train dtype:", X_train.dtype)
print("y_train dtype:", y_train.dtype)  
 

input_shape = (X_train.shape[1], X_train.shape[2])
model = build_lstm_model(input_shape)

  
print("X_train dtype:", X_train.dtype)     
print("y_train dtype:", y_train.dtype)
model.fit(X_train, y_train, epochs=5, batch_size=32)


Index(['Codigo', 'Descripcion', 'fechaHora', 'PrecEuro', 'Energia',
       'Categoria', 'Descripción', 'Agente', 'Porcentaje_Propiedad',
       'Tipo_Unidad', 'Zona/Frontera', 'Tecnología', 'log_Energia', 'Hora',
       'DiaSemana', 'Mes'],
      dtype='object')


ValueError: could not convert string to float: 'PREAL1'

In [85]:
print(df.dtypes)

cross_validate_model(df)

Codigo                          object
Descripcion                     object
fechaHora               datetime64[ns]
PrecEuro                       float64
Energia                        float64
Categoria                       object
Descripción                     object
Agente                          object
Porcentaje_Propiedad           float64
Tipo_Unidad                     object
Zona/Frontera                   object
Tecnología                      object
log_Energia                    float64
Hora                             int32
DiaSemana                        int32
Mes                              int32
dtype: object


c:\Users\annap\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\annap\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: Invalid dtype: object

## Forecasting

In [40]:


sequence_length = 24
X = df.drop(columns=['Energia', 'fechaHora'])  
y = df['Energia'] 
print("Shape of X before reshaping:", X.shape)
print("Shape of y before slicing:", y.shape)
X = X.select_dtypes(include=[np.number])


# numeric_columns = X.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


n_samples = X_scaled.shape[0] // sequence_length
X = X_scaled[:n_samples * sequence_length]
X = X.reshape((n_samples, sequence_length, X.shape[1]))
y = y.iloc[sequence_length-1:n_samples * sequence_length:sequence_length].values

print("Shape of X:", X.shape)  # Expected: (100396, 24, 9)
print("Shape of y:", y.shape)  # Expected: (100396,)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)

print("X_train dtype:", X_train.dtype)
print("y_train dtype:", y_train.dtype)


input_shape = (X_train.shape[1], X_train.shape[2])
model = build_lstm_model(input_shape)


print("X_train dtype:", X_train.dtype)
print("y_train dtype:", y_train.dtype)
model.fit(X_train, y_train, epochs=8, batch_size=32)

Shape of X before reshaping: (2409526, 10)
Shape of y before slicing: (2409526,)
Shape of X: (100396, 24, 2)
Shape of y: (100396,)
X_train dtype: float32
y_train dtype: float32
X_train dtype: float32
y_train dtype: float32
Epoch 1/8


c:\Users\annap\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2510/2510 ━━━━━━━━━━━━━━━━━━━━ 65s 25ms/step - loss: 5188.1152
Epoch 2/8
2510/2510 ━━━━━━━━━━━━━━━━━━━━ 68s 27ms/step - loss: 4878.4912
Epoch 3/8
2510/2510 ━━━━━━━━━━━━━━━━━━━━ 59s 23ms/step - loss: 5131.0024
Epoch 4/8
2510/2510 ━━━━━━━━━━━━━━━━━━━━ 70s 19ms/step - loss: 5916.4756
Epoch 5/8
2510/2510 ━━━━━━━━━━━━━━━━━━━━ 51s 20ms/step - loss: 6227.3403
Epoch 6/8
2510/2510 ━━━━━━━━━━━━━━━━━━━━ 51s 20ms/step - loss: 4918.7920
Epoch 7/8
2510/2510 ━━━━━━━━━━━━━━━━━━━━ 51s 20ms/step - loss: 5893.1675
Epoch 8/8
2510/2510 ━━━━━━━━━━━━━━━━━━━━ 51s 20ms/step - loss: 5689.7109


In [59]:
class ForecastingSequence(Sequence):
    def __init__(self, df, sequence_length=24, batch_size=32):
        self.df = df
        self.sequence_length = sequence_length
        self.batch_size = batch_size
        
        if 'Codigo' in df.columns:
            self.plants = df['Codigo'].unique()
        else:
            self.plants = ['default']  
        
    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    


    def __getitem__(self, index):
        if 'Codigo' in self.df.columns:
            plant_code= self.plants[index % len(self.plants)]
            plant_data= self.df[self.df['Codigo'] == plant_code]
        else:
            plant_data = self.df        #uses full dataset if no Codigo
            
        batch_features = []
        
        for idx in range(self.sequence_length, len(plant_data)):
            sequence = plant_data.iloc[idx - self.sequence_length:idx]
            features = sequence.drop(columns=['Energia', 'log_Energia', 'fechaHora'], errors='ignore')
            
            batch_features.append(features.values)
            # batch_features.append(features.values.reshape(self.sequence_length, -1))
        
        batch_features = np.array(batch_features)
        print(f"Shape of batch_features: {batch_features.shape}")
        return np.array(batch_features), np.zeros(len(batch_features))


future_dates = pd.date_range(start="2024-06-01", periods=29 * 24, freq='H')

df_future = pd.DataFrame({'fechaHora': future_dates})
df_future['Hora'] = df_future['fechaHora'].dt.hour
df_future['DiaSemana'] = df_future['fechaHora'].dt.weekday
df_future['Mes'] = df_future['fechaHora'].dt.month


df_future['PrecEuro'] = df_omie_labelled['PrecEuro'].mean()
print(df_future.columns)


scaler.fit(df_future[['PrecEuro', 'Hora', 'DiaSemana', 'Mes']])
df_future[['PrecEuro', 'Hora', 'DiaSemana', 'Mes']] = scaler.transform(df_future[['PrecEuro', 'Hora', 'DiaSemana', 'Mes']])

# df_future_reshaped = df_future[['PrecEuro', 'Hora', 'DiaSemana', 'Mes']].values
# df_future_reshaped = df_future_reshaped.reshape(df_future_reshaped.shape[0], 1, df_future_reshaped.shape[1])

forecast_generator = ForecastingSequence(df_future)
# predictions = model.predict(forecast_generator, verbose=0)
batch_features, _ = forecast_generator[0]  # Get a single batch
batch_features = batch_features.reshape(-1, forecast_generator.sequence_length, batch_features.shape[-1])  # Adjust this to the expected shape
print(f"Shape of batch_features for prediction: {batch_features.shape}")

predictions = model.predict(batch_features, verbose=0)
df_future['Forecast_Energia'] = np.expm1(predictions)

print(df_future[['fechaHora', 'Forecast_Energia']])


Index(['fechaHora', 'Hora', 'DiaSemana', 'Mes', 'PrecEuro'], dtype='object')
Shape of batch_features: (672, 24, 4)
(672, 24, 4)
Shape of batch_features: (672, 24, 4)


c:\Users\annap\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Shape of batch_features: (672, 24, 4)
Shape of batch_features: (672, 24, 4)


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 4 and 2 for '{{node sequential_7_1/lstm_14_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_7_1/lstm_14_1/strided_slice_1, sequential_7_1/lstm_14_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [672,4], [2,256].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(672, 4), dtype=float32)
  • states=('tf.Tensor(shape=(672, 64), dtype=float32)', 'tf.Tensor(shape=(672, 64), dtype=float32)')
  • training=False

Shape of batch_features: (672, 24, 4)


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_future['fechaHora'], df_future['Forecast_Energia'], label='Forecasted Energy Supply')

plt.xlabel('Time')
plt.ylabel('Forecasted Energy (MWh)')
plt.title('Energy Forecast for June 2024')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
df_future.to_csv("energy_forecast_june_2024.csv", index=False)

## Results: 

results_tanh_m (epochs=20, sequence_length = 24, batch_size = 64, LSTM units=[50, 50, 30], learning rate=0.01): 

{'Loss': 0.0001265329192392528,

 'MAE': 0.004819060364125532,

 'RMSE': 0.011248683749875237,

 'R2': 0.8340372729665887,
 
 'MAPE': inf}

{'Loss': 0.00010760701115941629,
 'MAE': 0.004284586236129825,
 'RMSE': 0.010373377932773709,
 'R2': 0.8588608329819016,
 'MAPE': 133.4399663867288,
 'Regression Accuracy': -33.4399663867288}